In [1]:
#Fill these file paths with the locations on your machine.
PATH_TO_PASTIS = 'D://content//PASTIS//PASTIS'



import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib
import os
matplotlib.use('agg')

# 构建colormap用于画图
cm = matplotlib.cm.get_cmap('tab20')
def_colors = cm.colors
cus_colors = ['k'] + [def_colors[i] for i in range(1,20)]+['w']   # 合并数组
cmap = ListedColormap(colors = cus_colors, name='agri',N=21)



def get_rgb(x):
    """Utility function to get a displayable rgb image 
    from a Sentinel-2 time series.
    """
    batch_index = 0
    im_list = []
    for t_show in range(x.shape[1]):
        im = x[batch_index, t_show, [2,1,0]].cpu().numpy() # to (128,128,10)
        mx = im.max(axis=(1,2))
        mi = im.min(axis=(1,2))   
        im = (im - mi[:,None,None])/(mx - mi)[:,None,None] # 归一化
        im = im.swapaxes(0,2).swapaxes(0,1)
        im = np.clip(im, a_max=1, a_min=0) # 将值限定在a_min到a_max之间
        im_list.append(im)
    return im_list

def save_rgb(x,dates,patch_id):
    """
    save rgb image
    """
    batch_index = 0
    for t_show in range(x.shape[1]):
        im = x[batch_index, t_show, [2,1,0]].cpu().numpy() # to (128,128,10)
        mx = im.max(axis=(1,2))
        mi = im.min(axis=(1,2))   
        im = (im - mi[:,None,None])/(mx - mi)[:,None,None] # 归一化
        im = im.swapaxes(0,2).swapaxes(0,1)
        im = np.clip(im, a_max=1, a_min=0) # 将值限定在a_min到a_max之间
        if not os.path.exists("pic/{}".format(patch_id)):
            os.mkdir("pic/{}".format(patch_id))
        plt.imsave("pic/{}/index{}date{}.jpg".format(patch_id,t_show,dates[batch_index][t_show]),im)
        

In [3]:
import sys
from dataloader_collect import PASTIS_Dataset
import collate
dt = PASTIS_Dataset(PATH_TO_PASTIS, norm=True, target='semantic')
# If you only need to evaluate semantic segmentation use target='semantic'


# Batch_size设置为1
# 用于显示每个patch的图像
dl = torch.utils.data.DataLoader(dt, batch_size=1, collate_fn=collate.pad_collate, shuffle=False)
print(dl.__len__())

Reading patch metadata . . .
Done.
Dataset ready.
2433


# 用作最后的图片筛选

In [10]:
s  = {40035,40058,40074,40128,40129,40497,40524}
for id, batch in enumerate(dl):

    (x, dates), y, item, id_patch = batch
    if id_patch.item() in s:
        pixel_semantic_annotation = y
        print("item{},id_patch{}".format(item[0],id_patch[0]))
        im = save_rgb(x,dates,patch_id=id_patch.item())

itemtensor([1912], dtype=torch.int32),id_patchtensor([40035])
itemtensor([1935], dtype=torch.int32),id_patchtensor([40058])
itemtensor([1951], dtype=torch.int32),id_patchtensor([40074])
itemtensor([2005], dtype=torch.int32),id_patchtensor([40128])
itemtensor([2006], dtype=torch.int32),id_patchtensor([40129])
itemtensor([2367], dtype=torch.int32),id_patchtensor([40497])
itemtensor([2394], dtype=torch.int32),id_patchtensor([40524])
